# Find all valuesets in differential of StructureDefinitions

- for hard coded path search all SD for binding and list the valuesets
- list em in a md table format in std out

In [1]:
from pathlib import Path
from json import load, dumps, loads
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from requests import get

headers = {'Accept': 'application/json+fhir',}
auth = ('apikey','5a96c64e-2156-43c0-a9b3-1c2c8d914bb5')
my_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/output')
data_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/input/data')
uscore_package_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev')
fhir_package_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1')
my_path, uscore_package_path, fhir_package_path

(PosixPath('/Users/ehaas/Documents/FHIR/US-Core/output'),
 PosixPath('/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev'),
 PosixPath('/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1'))

In [2]:
my_valuesets = set()
for i in my_path.glob('Struct*.json'):
    # print(i)
    obj = load(i.open())
    # print(obj['id'])
    #find reference type elements in diff
    for diff in obj['differential']['element']:
        # print(f"diff['id']= {diff['id']}")
        for snapshot in obj['snapshot']['element']:
            if snapshot['id'] == diff['id']:
                try:
                    # print(f'snapshot["binding"]["valueSet"] = {snapshot["binding"]["valueSet"]}')
                    my_valuesets.add(snapshot["binding"]["valueSet"])

                except KeyError:
                    # print(f"key error")
                    pass

my_valuesets


{'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1010.4',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1010.6',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1021.32',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.30',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.54',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1186.8',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113883.11.20.9.38',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.114222.4.11.1066',
 'http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.114222.4.11.3591',
 'http://hl7.org/fhir/ValueSet/address-use|4.0.1',
 'http://hl7.org/fhir/ValueSet/administrative-gender',
 'http://hl7.org/fhir/ValueSet/allergyintolerance-clinical',
 'http://hl7.org/fhir/ValueSet/allergyintolerance-verification',
 'http://hl7.org/fhir/ValueSet/care-plan-category',
 'http://hl7.org/fhir/ValueSet/care-plan-intent',
 'http://hl7.org/fhir/ValueSet/care

### get codesystems by looking up valuesets  package or VSAC


In [3]:
my_codesystems = set()

def get_vs_obj(vs_url,vs_filename):
    try:
        vs_file = fhir_package_path / 'package' / vs_filename
        vs_obj = loads(vs_file.read_text())
    except:
        try:
            vs_file = uscore_package_path / 'package' / vs_filename
            vs_obj = loads(vs_file.read_text())
        except:
            # print("-----------------")
            # print(vs)
            vs_obj = get(vs_url, headers=headers, auth=auth).json()
    return vs_obj

def get_system(vs_obj):
    for cs in vs_obj['compose']['include']:
        try:
            my_codesystems.add(cs['system'])
        except KeyError:
            # print(vs_obj['id'])
            # print("==============")
            # print(cs['valueSet'])
            for vs in cs['valueSet']:
                vs_filename = f"ValueSet-{vs.split('/')[-1].split('|')[0]}.json"
                # print(vs_filename)
                new_vs_obj = get_vs_obj(vs,vs_filename)
                # print("++++++++++++++++++++++")
                # print(new_vs_obj['id'], new_vs_obj['compose']['include'])
                get_system(new_vs_obj)

for i,vs in enumerate(my_valuesets):
    # print(vs)
    vs_filename = f"ValueSet-{vs.split('/')[-1].split('|')[0]}.json"
    vs_obj = get_vs_obj(vs,vs_filename)
    get_system(vs_obj)


my_codesystems


{'http://hl7.org/fhir/CodeSystem/medicationrequest-intent',
 'http://hl7.org/fhir/CodeSystem/medicationrequest-status',
 'http://hl7.org/fhir/address-use',
 'http://hl7.org/fhir/administrative-gender',
 'http://hl7.org/fhir/care-team-status',
 'http://hl7.org/fhir/contact-point-system',
 'http://hl7.org/fhir/contact-point-use',
 'http://hl7.org/fhir/diagnostic-report-status',
 'http://hl7.org/fhir/document-reference-status',
 'http://hl7.org/fhir/encounter-status',
 'http://hl7.org/fhir/event-status',
 'http://hl7.org/fhir/fm-status',
 'http://hl7.org/fhir/goal-status',
 'http://hl7.org/fhir/location-status',
 'http://hl7.org/fhir/name-use',
 'http://hl7.org/fhir/narrative-status',
 'http://hl7.org/fhir/observation-status',
 'http://hl7.org/fhir/questionnaire-answers-status',
 'http://hl7.org/fhir/request-intent',
 'http://hl7.org/fhir/request-status',
 'http://hl7.org/fhir/sid/cvx',
 'http://hl7.org/fhir/sid/icd-10-cm',
 'http://hl7.org/fhir/sid/icd-9-cm',
 'http://hl7.org/fhir/us/cor

### Write to file

In [4]:
out = data_path / 'codesystems.yml'
print( f'writing file to {out}')
out.write_text(y_dump(sorted(my_codesystems), default_flow_style=False, sort_keys=False))


writing file to /Users/ehaas/Documents/FHIR/US-Core/input/data/codesystems.yml


3145